<a href="https://colab.research.google.com/github/FrostLiam/GoogleExploreCSR_NPL_BERT/blob/main/DistilBERT_CSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Load dataset

#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

google_docs = {
    '1-HUEYGrKD67t-4Yc136ioyVztQSE8H7I' : 'fashion_dataset.csv', #raw data

    # after data cleaning (label, review)
    '1kmTVkueHUYtm1GzbKqOZg00_1OkZOz0V' :'fashion_2_train.csv', 
    '11mo70rOiutI73IG_7qvqcxBjHF2Wq69u' :'fashion_2_test.csv', 
}

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

for key in google_docs:
  drive.CreateFile({'id':key}).GetContentFile(google_docs[key])


In [3]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 130kB/s 
     |████████████████████████████████| 6.8MB 55.5MB/s 
     |████████████████████████████████| 983kB 49.8MB/s 
     |████████████████████████████████| 266kB 49.3MB/s 
     |████████████████████████████████| 1.9MB 46.8MB/s 
     |████████████████████████████████| 1.2MB 57.4MB/s 
     |████████████████████████████████| 471kB 43.0MB/s 
     |████████████████████████████████| 870kB 48.5MB/s 
     |████████████████████████████████| 3.3MB 44.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=864004c30db8866ed997569c61e75e470ead9f945730e2b0cae8ef70870348c8
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=2debac47996447f95410035d152f9d4779bd4c65c988eed5055f2ab1d9632831
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [4]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import ktrain
from ktrain import text

In [5]:
# read in training and test dataset
train_df = pd.read_csv('fashion_2_train.csv')
test_df = pd.read_csv('fashion_2_test.csv')

In [ ]:
train_df.info

<bound method DataFrame.info of        label                                             review
0          3  I had such high hopes for this dress and reall...
1          5  I love, love, love this jumpsuit. it's fun, fl...
2          5  This shirt is very flattering to all due to th...
3          2  I love tracy reese dresses, but this one is no...
4          5  I aded this in my basket at hte last mintue to...
...      ...                                                ...
13758      1  The styling of this top is so cute but the fab...
13759      3  I fell in love with the shirt when i saw it on...
13760      5  I tried on te regualr size (xs) as that is all...
13761      5  This sweater is substantial, well-made, fabric...
13762      4  Great top but it runs smaller than other tops ...

[13763 rows x 2 columns]>

In [6]:
# split both datasets into x and y based off of the review column and the label column
# mapped the labels to be strings instead of ints to bypass class_names which kept giving errors

x_train = list(train_df['review'])
y_train = list(map(str, train_df['label']))
x_test = list(test_df['review'])
y_test = list(map(str, test_df['label']))

In [7]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test,
                                          preprocess_mode="distilbert", lang='en')


preprocessing train...
language: en
train sequence lengths:
	mean : 62
	95percentile : 102
	99percentile : 106


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 63
	95percentile : 102
	99percentile : 106


task: text classification


In [9]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)

Is Multi-Label? False
maxlen is 400



done.


In [10]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [14]:
learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
2294/2294 [==============================] - 674s 293ms/step - loss: 0.8595 - accuracy: 0.6421 - val_loss: 0.9279 - val_accuracy: 0.6089
Epoch 2/4
2294/2294 [==============================] - 672s 292ms/step - loss: 0.7310 - accuracy: 0.6905 - val_loss: 0.8435 - val_accuracy: 0.6577
Epoch 3/4
2294/2294 [==============================] - 671s 292ms/step - loss: 0.6138 - accuracy: 0.7451 - val_loss: 0.8126 - val_accuracy: 0.6652
Epoch 4/4
2294/2294 [==============================] - 671s 291ms/step - loss: 0.4019 - accuracy: 0.8472 - val_loss: 0.9493 - val_accuracy: 0.6616


In [15]:
learner.validate()

              precision    recall  f1-score   support

           0       0.49      0.33      0.39       224
           1       0.36      0.33      0.34       422
           2       0.43      0.46      0.44       744
           3       0.48      0.50      0.49      1332
           4       0.85      0.85      0.85      3177

    accuracy                           0.66      5899
   macro avg       0.52      0.49      0.50      5899
weighted avg       0.66      0.66      0.66      5899



array([[  74,   88,   49,   11,    2],
       [  50,  138,  171,   56,    7],
       [  27,  132,  343,  212,   30],
       [   0,   23,  199,  662,  448],
       [   1,    2,   37,  451, 2686]])

In [17]:
p = ktrain.get_predictor(model, preproc)

In [19]:
p.get_classes()

['1', '2', '3', '4', '5']

In [21]:
# actual ratings were [4, 1, 5, 2, 5, 2]
p.predict(x_test[0:6])

['4', '2', '5', '2', '5', '1']

-----END OF CODE-----